# 暗示的怒り

In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# print(os.environ['CUDA_LAUNCH_BLOCKING'])



In [1]:
import torch
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

# GPUが使えれば利用する設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
import pandas as pd
dataset = pd.read_json("ksl_pub/suggestive_anger/data/fuman_survey/json/human_data_ver5.json")
print(f'データサイズ： {dataset.shape}')

# for i in range(len(dataset.fulltext.values)):
#     print(dataset.label.values[i],  dataset.fulltext.values[i])

# データの抽出
sentences = dataset.fulltext.values
labels = dataset.label.values

dataset.sample(3)

データサイズ： (1800, 5)


,fulltext,sentences,danwa_result,result_form,label
197,高い、狭い、古い。なんとかならないのか。,"[高い、狭い、古い, なんとかならないのか]",並列,"{'1': '3.0', '0': '8.0'}",0
1257,1000円以上で小冊子スタッフレシピをプレゼント。正直、要らない。,"[1000円以上で小冊子スタッフレシピをプレゼント, 正直、要らない]",詳細化,"{'1': '4.0', '3': '4.0', '2': '1.0'}",3
1372,日本の女子バスケはへたくそすぎる。特にとかしき!背がたかいだけでなんも決まんない!3ポイント...,"[日本の女子バスケはへたくそすぎる, 特にとかしき!背がたかいだけでなんも決まんない!3ポイ...",対比,"{'0': '3.0', '3': '2.0', '1': '4.0'}",1


In [4]:
# 1. BERT Tokenizerを用いて単語分割・IDへ変換
## Tokenizerの準備
# from transformers import BertJapaneseTokenizer
# tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
from transformers import T5Tokenizer, RobertaForMaskedLM
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('rinna/japanese-roberta-base')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
## テスト実行
# 元文章
for i in range(3):
    print('Original: ', sentences[i])
    print('Tokenized: ', tokenizer.tokenize(sentences[i]))    # Tokenizer
    print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[i])))    # Token-id
    print("\n\n")

In [ ]:
# 最大単語数の確認
max_len = []
# 1文づつ処理
for sent in sentences:
    # Tokenizeで分割
    token_words = tokenizer.tokenize(sent)
    # 文章数を取得してリストへ格納
    max_len.append(len(token_words))
# 最大の値を確認
print('最大単語数: ', max(max_len))
print('上記の最大単語数にSpecial token（[CLS], [SEP]）の+2をした値(' + str(max(max_len) + 2) + ')が最大単語数')

In [ ]:
import torch

input_ids = []
attention_masks = []

# 1文づつ処理
for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, # Special Tokenの追加
                        # max_length = 187,           # 文章の長さを固定（Padding/Trancatinating）
                        max_length = max(max_len) + 2,           # 文章の長さを固定（Padding/Trancatinating）
                        pad_to_max_length = True,# PADDINGで埋める
                        return_attention_mask = True,   # Attention maksの作成
                        return_tensors = 'pt',     #  Pytorch tensorsで返す
                   )

    # 単語IDを取得    
    input_ids.append(encoded_dict['input_ids'])

    # Attention　maskの取得
    attention_masks.append(encoded_dict['attention_mask'])

# リストに入ったtensorを縦方向（dim=0）へ結合
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# tenosor型に変換
labels = torch.tensor(labels, device=device)

# 確認
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# データセットクラスの作成
dataset = TensorDataset(input_ids, attention_masks, labels)

# 90%地点のIDを取得
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# データセットを分割
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('訓練データ数：{}'.format(train_size))
print('検証データ数:　{} '.format(val_size))

# データローダーの作成
batch_size = 32

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# BertForSequenceClassification 学習済みモデルのロード
model = BertForSequenceClassification.from_pretrained(
    "cl-tohoku/bert-base-japanese-whole-word-masking", # 日本語Pre trainedモデルの指定
    num_labels = 6, # ラベル数
    output_attentions = False, # アテンションベクトルを出力するか
    output_hidden_states = False, # 隠れ層を出力するか
).to(device)

# モデルをGPUへ転送
# 下の方でエラーが発生するため、一時的に無効化
model.cuda()


In [ ]:
# # 最適化手法の設定
# optimizer = AdamW(model.parameters(), lr=2e-5)

# # 訓練パートの定義
# def train(model):
#     model.train() # 訓練モードで実行
#     train_loss = 0
#     for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
#         b_input_ids = batch[0].to(device)
#         b_input_mask = batch[1].to(device)
#         b_labels = batch[2].to(device)
#         optimizer.zero_grad()
#         loss, logits = model(b_input_ids, 
#                              token_type_ids=None, 
#                              attention_mask=b_input_mask, 
#                              labels=b_labels)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
#         optimizer.step()
#         train_loss += loss.item()
#     return train_loss

# # テストパートの定義
# def validation(model):
#     model.eval()# 訓練モードをオフ
#     val_loss = 0
#     with torch.no_grad(): # 勾配を計算しない
#         for batch in validation_dataloader:
#             b_input_ids = batch[0].to(device)
#             b_input_mask = batch[1].to(device)
#             b_labels = batch[2].to(device)
#             with torch.no_grad():        
#                 (loss, logits) = model(b_input_ids, 
#                                     token_type_ids=None, 
#                                     attention_mask=b_input_mask,
#                                     labels=b_labels)
#             val_loss += loss.item()
#     return val_loss

In [ ]:
# 最適化手法の設定
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0
    for batch in train_dataloader: # train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        loss = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels).loss # 戻り値とここを修正

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval() # 訓練モードをオフ
    val_loss = 0
    with torch.no_grad(): # 勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                loss = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels).loss # 戻り値とここを修正
            val_loss += loss.item()
    return val_loss

In [ ]:
# 学習の実行
max_epoch = 1
train_loss_ = []
test_loss_ = []

for epoch in range(max_epoch):
    train_ = train(model)
    test_ = train(model)
    train_loss_.append(train_)
    test_loss_.append(test_)
train_loss_

In [ ]:
# 検証方法の確認（1バッチ分で計算ロジックに確認）

model.eval()# 訓練モードをオフ
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

In [ ]:
## 予測結果の確認
print(f'出力:{preds}')

In [ ]:
# 比較しやすい様にpd.dataframeへ整形
import pandas as pd
import numpy as np
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['感情的攻撃', '感情的説得', '理性的説得', '嫌味・皮肉', '遠回し', '怒ってない'])
## np.argmaxで大き方の値を取得
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label'])
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label'])

accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

accuracy_df.head()

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name())
# > Tesla T4
print(torch.cuda.get_device_capability())
# > (7, 5)


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# 変数を用意
a = torch.randn(5)
b = torch.randn(5)

# 今回は変数aだけをGPUに送る。
a = a.to(device)
b = b.to(device)

print(a + b)